
# 1. OpenAI VLM (GPT) - Basics
This section demonstrates the basic usage of OpenAI's Vision Language Model (VLM) capabilities using GPT-4.1.
We will use the OpenAI API to analyze an image and provide detailed textual insights.

**Support Material**

- https://platform.openai.com/docs/quickstart
- https://platform.openai.com/docs/guides/text
- https://platform.openai.com/docs/guides/images-vision?api-mode=chat
- https://platform.openai.com/docs/guides/structured-outputs


## Erklärung zu Code-Block (Zelle 1)

**Was passiert hier?**
- - Es werden Hilfsbibliotheken geladen und eine Funktion `encode_image(...)` definiert.
- - `encode_image` liest eine Bilddatei als Bytes ein und kodiert sie als Base64-String.
- - Anschließend wird `.env` geladen, ein OpenAI-Client erstellt und ein Bildpfad gesetzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Base64 wird genutzt, um ein lokales Bild als String direkt in einer API-Anfrage zu übertragen.
- - Client + Bildpfad sind die Voraussetzung für spätere Vision-Calls (Text + Bild).

**Rolle im Gesamtprozess**
Vorbereitung: Hilfsfunktion + Client/Inputs bereitstellen.

**Wichtig / typische Stolpersteine (optional)**
- Wenn `img`-Pfad falsch ist, schlägt das Einlesen/Kodieren später fehl.


In [3]:
import openai
from dotenv import load_dotenv  
import base64
import json
import textwrap

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')  # Bildbytes → Base64 kodieren


load_dotenv()  # lädt Variablen aus .env
openAIclient = openai.OpenAI()  # API-Client initialisieren


# Path to your image
img = "images/street_scene.jpg"




## Erklärung zu Code-Block (Zelle 2)

**Was passiert hier?**
- - Es wird eine Chat-Completion mit einem Vision-fähigen Modell aufgerufen.
- - In `messages` wird Text + Bild gemeinsam als Input gesendet (multimodal).
- - Die Antwort wird aus dem Response-Objekt als Text extrahiert und zur Lesbarkeit umbrochen ausgegeben.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Typischer „Basic VLM Call“: schnell prüfen, ob das Modell das Bild korrekt beschreiben kann.
- - Grundlage für spätere strukturierte Extraktion (z.B. JSON) oder Aufgaben wie Objekt-/Szenenbeschreibung.

**Rolle im Gesamtprozess**
LLM/VLM-Call: Bild + Prompt → Modellantwort anzeigen.

**Wichtig / typische Stolpersteine (optional)**
- `openAIclient` und `img` müssen vorher gesetzt sein (sonst Fehler).
- Response ist ein Objekt; relevant ist meist `choices[0].message.content`.


In [4]:
#basic call to gpt with prompt and image

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",  # ausgewähltes Modell
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",  # Bild als Base64-Data-URL mitsenden
                    "image_url": {  # Bild als Base64-Data-URL mitsenden
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))  # Ausgabe umbrechen (lesbarer)


The image shows a busy urban street scene with a mixture of people engaged in various activities. In the foreground,
there is a young person sitting on the ground using a tablet or phone next to a potted plant with orange flowers.
Nearby, another person is lying down on the pavement, wearing a red jacket and a beanie. On a bench, an older man in a
suit is sitting next to a young woman with blonde hair, who is reading a newspaper.   In the background, there are cars
moving through a crosswalk, a motorcyclist, a man walking and playing a guitar, and a scooter rider. A young woman is
walking on the sidewalk while looking at her phone. Several pigeons are scattered on the ground near the bench and the
person lying down.  The scene is set against a backdrop of tall buildings and city infrastructure, with a warm, late
afternoon or early evening light. The image captures a vibrant, dynamic urban life moment with a mix of technology,
transportation, and leisurely activities.



# 1.1 Structured Output
Here, we expand upon the VLM example to request structured outputs. This approach allows for extracting 
well-organized information from images in a machine-readable format, such as JSON.

**Support Material**:
- https://platform.openai.com/docs/guides/structured-outputs


## Erklärung zu Code-Block (Zelle 3)

**Was passiert hier?**
- - Es wird erneut ein Vision-Call gemacht, diesmal mit System-Anweisung für vorsichtige Beobachtung.
- - Mit `response_format` wird die Antwort explizit als JSON-Objekt angefordert.
- - Der JSON-Text wird aus der Antwort herausgegriffen und in `returnValue` gespeichert.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Strukturierte Outputs (JSON) sind im DS-Kontext leichter zu parsen, prüfen und weiterzuverarbeiten.
- - Reduziert das Risiko von „freiem Text“, wenn du später auf einzelne Felder zugreifen willst.

**Rolle im Gesamtprozess**
LLM/VLM-Call mit Formatvorgabe: strukturierte Bildbeschreibung erzeugen.

**Wichtig / typische Stolpersteine (optional)**
- Wenn das Modell kein gültiges JSON liefert, kann `json.loads(...)` später fehlschlagen.
- `temperature=0` unterstützt reproduzierbarere, stabilere Struktur.


In [46]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",  # ausgewähltes Modell
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",  # Bild als Base64-Data-URL mitsenden
                    "image_url": {  # Bild als Base64-Data-URL mitsenden
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={ "type": "json_object" },# NEW!!
    temperature = 0  # Kreativität/Determinismus steuern
)

returnValue = completion.choices[0].message.content  # Antworttext aus Response extrahieren


## Erklärung zu Code-Block (Zelle 4)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [47]:
returnValue

'{\n  "scene": "Urban city street intersection during daytime",\n  "background": {\n    "buildings": [\n      {\n        "style": "Brick facade with green awnings",\n        "windows": "Multiple, reflecting sunlight",\n        "shops": "Visible with warm lighting inside"\n      },\n      {\n        "style": "Modern glass skyscrapers",\n        "height": "Tall, extending into the sky",\n        "details": "Reflective surfaces with some trees in front"\n      },\n      {\n        "church": {\n          "architecture": "Traditional with a steeple",\n          "location": "Center background"\n        }\n      }\n    ],\n    "traffic_light": {\n      "color": "Yellow",\n      "position": "Right side above the street"\n    }\n  },\n  "street": {\n    "crosswalk": "Wide zebra stripes",\n    "vehicles": [\n      {\n        "type": "Taxi",\n        "color": "White",\n        "motion": "Blurred, indicating movement"\n      },\n      {\n        "type": "SUV",\n        "color": "Gray",\n        "m

We parse the json in a dict structure:

## Erklärung zu Code-Block (Zelle 5)

**Was passiert hier?**
- - Ein JSON-String wird in ein Python-Objekt (meist dict/list) umgewandelt.
- - Anschließend wird das Objekt angezeigt/weiter genutzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Damit man Felder gezielt auslesen kann (z.B. `output['background']...`).
- - Ermöglicht einfache Weiterverarbeitung wie Auswertungen/Regeln/Alerts.

**Rolle im Gesamtprozess**
Auswertung: Antwort strukturieren, damit man damit arbeiten kann.

**Wichtig / typische Stolpersteine (optional)**
- Wenn der String kein valides JSON ist, gibt es einen Parsing-Fehler.


In [18]:
output = json.loads(returnValue)  # JSON-String → Python-Objekt
#json. loads() converts JSON strings to Python objects
print(output)

{'scene': 'Urban city street with a mix of pedestrians, vehicles, and street furniture', 'time_of_day': 'Late afternoon or early evening, indicated by the warm sunlight and long shadows', 'background': {'buildings': [{'style': 'Brick facade with large windows and green awnings', 'location': 'Left side of the street'}, {'style': 'Modern glass and steel skyscrapers', 'location': 'Center and right side, extending into the distance'}, {'style': 'Historic church-like building with a steeple', 'location': 'Center background'}], 'traffic_lights': {'color': 'Yellow', 'position': 'Hanging over the street on the right side'}, 'street': {'type': 'Crosswalk with white zebra stripes', 'vehicles': [{'type': 'Taxi', 'color': 'White', 'motion': 'Blurred, indicating movement', 'location': 'Left side of the crosswalk'}, {'type': 'SUV', 'color': 'Gray', 'motion': 'Moving', 'location': 'Behind the taxi'}, {'type': 'Sedan', 'color': 'Orange', 'motion': 'Moving', 'location': 'Center of the crosswalk'}], 'mo

So we can access specific infos:

## Erklärung zu Code-Block (Zelle 6)

**Was passiert hier?**
- - Es wird ein konkretes Feld aus dem JSON-Objekt per Key/Index herausgegriffen.
- - Das ausgewählte Detail wird ausgegeben.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Prüft, ob die strukturierte Extraktion so aufgebaut ist wie erwartet.
- - In RAG/VLM-Pipelines oft Schritt zur Validierung von Schema/Parsing.

**Rolle im Gesamtprozess**
Auswertung/Validierung: gezielt ein Ergebnisfeld prüfen.

**Wichtig / typische Stolpersteine (optional)**
- KeyError/IndexError, wenn das JSON-Schema anders ist als angenommen.


In [25]:
print(output["background"]["street"]["vehicles"][1]["location"])


Behind the taxi



# JSON Schema for Controlled Structured Outputs
In this section, we define a JSON schema for a more controlled and specific output from the model. 
Using this schema, we can ensure the model adheres to predefined data types and structures while describing images.In this case we will provide the json schema directly.



## Erklärung zu Code-Block (Zelle 7)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [23]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",  # ausgewähltes Modell
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",  # Bild als Base64-Data-URL mitsenden
                    "image_url": {  # Bild als Base64-Data-URL mitsenden
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={  # Ausgabeformat erzwingen (JSON)
                "type": "json_schema",    
                "json_schema": {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]
                    }}},
    temperature = 0  # Kreativität/Determinismus steuern
)

returnValue = completion.choices[0].message.content  # Antworttext aus Response extrahieren


## Erklärung zu Code-Block (Zelle 8)

**Was passiert hier?**
- - Ein JSON-String wird in ein Python-Objekt (meist dict/list) umgewandelt.
- - Anschließend wird das Objekt angezeigt/weiter genutzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Damit man Felder gezielt auslesen kann (z.B. `output['background']...`).
- - Ermöglicht einfache Weiterverarbeitung wie Auswertungen/Regeln/Alerts.

**Rolle im Gesamtprozess**
Auswertung: Antwort strukturieren, damit man damit arbeiten kann.

**Wichtig / typische Stolpersteine (optional)**
- Wenn der String kein valides JSON ist, gibt es einen Parsing-Fehler.


In [26]:
output_image_extraction = json.loads(returnValue)  # JSON-String → Python-Objekt


## Erklärung zu Code-Block (Zelle 9)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [27]:
output_image_extraction["people"]

[{'position': 'sitting on the sidewalk',
  'age': 15,
  'activity': 'using a smartphone',
  'gender': 'male'},
 {'position': 'lying on the sidewalk',
  'age': 20,
  'activity': 'resting or sleeping',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 65,
  'activity': 'thinking or resting',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 25,
  'activity': 'reading a newspaper',
  'gender': 'female'},
 {'position': 'walking on the sidewalk',
  'age': 20,
  'activity': 'using a smartphone',
  'gender': 'female'},
 {'position': 'riding a motorcycle',
  'age': 30,
  'activity': 'driving',
  'gender': 'male'},
 {'position': 'walking on the street',
  'age': 30,
  'activity': 'playing guitar',
  'gender': 'male'},
 {'position': 'riding a scooter',
  'age': 25,
  'activity': 'driving',
  'gender': 'female'},
 {'position': 'walking on the sidewalk in the background',
  'age': 30,
  'activity': 'walking',
  'gender': 'female'},
 {'position': 'walking on the side

Alternatively: 


OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively. Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.


```python
from pydantic import BaseModel


class Person(BaseModel):
    position: str 
    age: int 
    activity: str 
    gender: str


class ImageExtraction(BaseModel):
    number_of_people: int 
    atmosphere: str 
    hour_of_the_day: int 
    people: list[Person] 

completion = openAIclient.beta.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": "describe the image in detail"}
    ],
    response_format=ImageExtraction,
)

output_image_extraction = completion.choices[0].message.parsed


We can then integrate the extracted information in full or partially in a new prompt for a new extraction

## Erklärung zu Code-Block (Zelle 10)

**Was passiert hier?**
- - Es wird ein Prompt für einen nachgelagerten Schritt (z.B. Alert/Entscheidung) zusammengestellt.
- - Die vorher extrahierten Bildinformationen werden in den Prompt eingebettet.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Post-Processing: aus Roh-Extraktion werden handlungsorientierte Outputs (z.B. Warnungen, Zusammenfassungen).
- - Typisch im DS-Kontext: Regeln/Business-Logik über LLM-Output legen.

**Rolle im Gesamtprozess**
Weiterverarbeitung: aus Extraktion wird ein Folge-Prompt/Task.

**Wichtig / typische Stolpersteine (optional)**
- Zu lange Prompts können Token-Limits auslösen; evtl. nur relevante Teile übergeben.


In [28]:
#alert service prompt 

alert_sys_prompt = " you are an experienced first aid paramedical"
alert_prompt= """Extract from the following scene analysis give to you in json format, 
if anyone might be in danger and if the Child Hospital or normal Hospital should be alerted. 
Give the a concise answer
The situation is given to you from this object: """ + str(output_image_extraction)


## Erklärung zu Code-Block (Zelle 11)

**Was passiert hier?**
- - Es wird ein Prompt für einen nachgelagerten Schritt (z.B. Alert/Entscheidung) zusammengestellt.
- - Die vorher extrahierten Bildinformationen werden in den Prompt eingebettet.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Post-Processing: aus Roh-Extraktion werden handlungsorientierte Outputs (z.B. Warnungen, Zusammenfassungen).
- - Typisch im DS-Kontext: Regeln/Business-Logik über LLM-Output legen.

**Rolle im Gesamtprozess**
Weiterverarbeitung: aus Extraktion wird ein Folge-Prompt/Task.

**Wichtig / typische Stolpersteine (optional)**
- Zu lange Prompts können Token-Limits auslösen; evtl. nur relevante Teile übergeben.


In [29]:

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",  # ausgewähltes Modell
    messages=[
        {"role": "user", "content": alert_prompt},
        {"role": "user", "content": alert_prompt}
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))  # Ausgabe umbrechen (lesbarer)


No one appears to be in immediate danger. The 20-year-old lying on the sidewalk seems to be resting or sleeping without
signs of distress. No hospital alert is necessary. If needed, a normal hospital should be contacted rather than a child
hospital.


## Erklärung zu Code-Block (Zelle 12)

**Was passiert hier?**
- - Es wird eine Chat-Completion mit einem Vision-fähigen Modell aufgerufen.
- - In `messages` wird Text + Bild gemeinsam als Input gesendet (multimodal).
- - Die Antwort wird aus dem Response-Objekt als Text extrahiert und zur Lesbarkeit umbrochen ausgegeben.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Typischer „Basic VLM Call“: schnell prüfen, ob das Modell das Bild korrekt beschreiben kann.
- - Grundlage für spätere strukturierte Extraktion (z.B. JSON) oder Aufgaben wie Objekt-/Szenenbeschreibung.

**Rolle im Gesamtprozess**
LLM/VLM-Call: Bild + Prompt → Modellantwort anzeigen.

**Wichtig / typische Stolpersteine (optional)**
- `openAIclient` und `img` müssen vorher gesetzt sein (sonst Fehler).
- Response ist ein Objekt; relevant ist meist `choices[0].message.content`.


In [30]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",  # ausgewähltes Modell
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Considering this list of people"+str(output_image_extraction["people"])+".Identify the youngest in the picture I provide and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."},
                {
                    "type": "image_url",  # Bild als Base64-Data-URL mitsenden
                    "image_url": {  # Bild als Base64-Data-URL mitsenden
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))  # Ausgabe umbrechen (lesbarer)


The youngest person according to the list is a 15-year-old male sitting on the sidewalk using a smartphone. In the
provided image, the person corresponding to this description is the boy sitting at the bottom left corner of the image,
holding a phone or tablet.  The approximate coordinates of this person in [ymin, xmin, ymax, xmax] format normalized to
a 0-1000 scale would be:  [ymin: 635, xmin: 130, ymax: 880, xmax: 280]



# 2. Google VLM (Gemini)
This section demonstrates the use of Google's Vision Language Model, Gemini. 
We explore basic text generation as well as its ability to analyze images and provide relevant outputs.

**Support Material**:
- https://ai.google.dev/gemini-api/docs/quickstart
- https://ai.google.dev/gemini-api/docs/text-generation
- https://ai.google.dev/gemini-api/docs/image-understanding
- https://ai.google.dev/gemini-api/docs/structured-output?example=recipe

## Erklärung zu Code-Block (Zelle 13)

**Was passiert hier?**
- Imports & Setup: Es werden Bibliotheken eingebunden, die für VLM/LLM-Calls, Bildhandling und Ausgabe gebraucht werden.
- `from dotenv import load_dotenv` — Umgebungsvariablen (.env) laden.
- `from google import genai` — Gemini-API (Google GenAI).
- `from PIL import Image` — Bilder laden/verarbeiten.
- `import textwrap` — lange Ausgaben umbrechen.
- `import json` — JSON einlesen/ausgeben.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- Damit alle Bausteine bereitstehen, um Bilder + Text als Input an ein VLM zu schicken und strukturierte Outputs auszuwerten.

**Rolle im Gesamtprozess**
Setup: vorbereitet die Arbeitsumgebung, bevor Modellaufrufe und Auswertung passieren.


In [31]:
%matplotlib inline
from dotenv import load_dotenv  
from google import genai
from PIL import Image
import textwrap

import json


load_dotenv()  # lädt Variablen aus .env
client = genai.Client()

# Path to your image
img = "images/street_scene.jpg"

Basic call:

## Erklärung zu Code-Block (Zelle 14)

**Was passiert hier?**
- - Es wird ein Gemini-Client genutzt, um eine (Vision-)Anfrage zu senden.
- - Textprompt und ggf. Bild werden als Input übergeben.
- - Die Antwort wird ausgegeben (mit `textwrap` umbrochen).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Zeigt, wie man Vision-Modelle auch über Gemini im Notebook testet.
- - Gut für Modellvergleich: gleicher Task, anderer Provider.

**Rolle im Gesamtprozess**
LLM/VLM-Call (Gemini): Prompt/Bild → Antwort.

**Wichtig / typische Stolpersteine (optional)**
- API-Key/Client muss korrekt konfiguriert sein; sonst Auth-Fehler.


In [32]:
response = client.models.generate_content(  # LLM/VLM-Call an Gemini
    model="gemini-2.5-flash", contents="Explain how AI works to a 90 years old. in few words"  # ausgewähltes Modell
)

print(textwrap.fill(response.text, width=120))  # Ausgabe umbrechen (lesbarer)


Imagine a very smart computer helper.  You show it many, many examples, and it learns to recognize patterns so it can do
things like understand your voice or spot faces in photos.


and with images: 

## Erklärung zu Code-Block (Zelle 15)

**Was passiert hier?**
- - Es werden Bibliotheken für Bildverarbeitung/Visualisierung geladen und ein Bild geöffnet.
- - Danach wird eine Gemini-Anfrage mit Bild + Textprompt vorbereitet/abgeschickt (falls enthalten).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - VLM-Experimente brauchen Bild-Input; PIL/Matplotlib helfen beim Laden und Prüfen der Inputs.
- - Visualisierung unterstützt Debugging (ist das richtige Bild geladen?).

**Rolle im Gesamtprozess**
Vorbereitung/Debugging: Bild laden und ggf. für VLM-Call bereitstellen.


In [33]:
im = Image.open(img)  # Bilddatei öffnen

response = client.models.generate_content(model="gemini-2.5-flash",  # ausgewähltes Modell
                                          contents=[im, "Describe the scene in details\n"],
                                          )

print(textwrap.fill(response.text, width=120))  # Ausgabe umbrechen (lesbarer)


This bustling urban scene captures a moment in a vibrant city during what appears to be late afternoon or early evening,
bathed in the warm, golden light of a low sun. The image is rich with activity, showcasing a blend of architectural
styles and diverse individuals going about their day.  In the **foreground**, a wide pedestrian crosswalk with bold
black and white stripes diagonally cuts across the bottom left. On the sidewalk to the left, a small pot of red
geraniums sits. Next to it, a young person with short brown hair sits cross-legged, engrossed in a tablet or phone.
Further to the right and slightly in front, another young person, wearing a red hoodie and blue jeans, lies casually on
their back on the pavement, looking upwards. Several pigeons are scattered on the sidewalk and crosswalk, pecking at the
ground, adding to the authentic urban feel.  On the right side of the foreground, a classic wooden park bench is
occupied by two individuals. An older man in a dark suit sits on 

Also here we can extract structured output (Gemini actually prefers pydantic syntax - let's see what happens with a schema as before)-> check limitations in https://ai.google.dev/gemini-api/docs/structured-output?example=recipe 

## Erklärung zu Code-Block (Zelle 16)

**Was passiert hier?**
- - Es wird ein Gemini-Client genutzt, um eine (Vision-)Anfrage zu senden.
- - Textprompt und ggf. Bild werden als Input übergeben.
- - Die Antwort wird ausgegeben (mit `textwrap` umbrochen).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Zeigt, wie man Vision-Modelle auch über Gemini im Notebook testet.
- - Gut für Modellvergleich: gleicher Task, anderer Provider.

**Rolle im Gesamtprozess**
LLM/VLM-Call (Gemini): Prompt/Bild → Antwort.

**Wichtig / typische Stolpersteine (optional)**
- API-Key/Client muss korrekt konfiguriert sein; sonst Auth-Fehler.


In [34]:
json_schema = {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]}}



config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema,
    }


response = client.models.generate_content(model="gemini-2.5-flash",  # ausgewähltes Modell
                                          contents=[im, "Describe the scene in details, follwoing exactly the given json schema\n"],
                                          config=config
                                          )



print(response.text)

{
  "visual_description": "A dynamic street scene in a bustling city, featuring a mix of people, vehicles, and architecture under a soft, golden hour light. The foreground shows a wide crosswalk where several individuals are engaged in various activities, while cars and motorcycles move past. Tall buildings, ranging from classic brick structures to modern skyscrapers, line the street, creating a deep urban perspective. Pigeons are scattered on the sidewalk, and a single potted plant adds a touch of nature.",
  "elements": [
    "crosswalk",
    "traffic light",
    "street lamps",
    "buildings",
    "skyscrapers",
    "cars",
    "motorcycle",
    "scooter",
    "benches",
    "pigeons",
    "potted plant with red flowers",
    "sidewalk"
  ],
  "people": [
    "A man in a dark jacket and helmet riding a motorcycle across the crosswalk.",
    "A man in a dark jacket and hat walking and playing a guitar across the crosswalk.",
    "A woman riding a scooter in the background, to the ri

Does it match your schema?

Let's try to use Gemini to detect an object in the image and get its coordinates:


## Erklärung zu Code-Block (Zelle 17)

**Was passiert hier?**
- - Ein JSON-String wird in ein Python-Objekt (meist dict/list) umgewandelt.
- - Anschließend wird das Objekt angezeigt/weiter genutzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Damit man Felder gezielt auslesen kann (z.B. `output['background']...`).
- - Ermöglicht einfache Weiterverarbeitung wie Auswertungen/Regeln/Alerts.

**Rolle im Gesamtprozess**
Auswertung: Antwort strukturieren, damit man damit arbeiten kann.

**Wichtig / typische Stolpersteine (optional)**
- Wenn der String kein valides JSON ist, gibt es einen Parsing-Fehler.


In [ ]:
prompt = "Identify the youngest in the picture and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."


config={"response_mime_type": "application/json"}

response = client.models.generate_content(model="gemini-2.5-flash",  # ausgewähltes Modell
                                          contents=[img, prompt],
                                          config=config
                                          )

bounding_boxes = json.loads(response.text)  # JSON-String → Python-Objekt
print(bounding_boxes)


{'box_2d': [664, 461, 794, 513]}
{
  "box_2d": [664, 461, 794, 513]
}


Gemini2+ was trained specifically for object detection/ segmentation tasks. More details: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Spatial_understanding.ipynb


## 3.Extract Structured Infos from Hand-written note - GPT & Gemini

Let’s try **not** to extract structured information from a handwritten note (e.g., `prescription1.jpg`) using **both models**.

Consider the file: `/images/prescription1.jpg`.  
Have a look at it.

### JSON Schema
Let’s define a JSON schema for the extraction task:


## Erklärung zu Code-Block (Zelle 18)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [36]:
json_schema_prescription = {
 "name": "prescription_extract",
"schema": {
  "type": "object",
  "properties": {
    "doctor_name": { "type": "string" },
    "patient_name": { "type": "string" },
    "patient_dob": { "type": "string" },
    "meds": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "dose": { "type": "string" },
          "frequency": { "type": "string" },
          "instructions": { "type": "string" }
        },
        "required": ["name"]
      }
    },
    "signature": { "type": "boolean" }
  },
  "required": ["doctor_name", "patient_name", "meds"]
}}

Extract structured infos using Gemini: 

## Erklärung zu Code-Block (Zelle 19)

**Was passiert hier?**
- - Es werden Bibliotheken für Bildverarbeitung/Visualisierung geladen und ein Bild geöffnet.
- - Danach wird eine Gemini-Anfrage mit Bild + Textprompt vorbereitet/abgeschickt (falls enthalten).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - VLM-Experimente brauchen Bild-Input; PIL/Matplotlib helfen beim Laden und Prüfen der Inputs.
- - Visualisierung unterstützt Debugging (ist das richtige Bild geladen?).

**Rolle im Gesamtprozess**
Vorbereitung/Debugging: Bild laden und ggf. für VLM-Call bereitstellen.


In [45]:
im = Image.open("images/prescription1.jpg")  # Bilddatei öffnen

config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema_prescription,
    }


response = client.models.generate_content(model="gemini-2.5-flash",  # ausgewähltes Modell
                                          contents=[im, "Extract infos from image, follwoing the given json schema.\n"],
                                          config=config
                                          )



print(response.text)

{
  "doctor": "Dr. Markus Müller",
  "patient": "Claudle Fischer",
  "dateOfBirth": "01.04.1978",
  "gender": "Female",
  "medication": [
    "Ibuprofen",
    "3x 400mg",
    "nach dem Essen"
  ],
  "signature": "Reptuller"
}


If the output is **not valid JSON** and contains extra strings, it must be **parsed** before it can be loaded into a Python dict.  
Below is an example helper function that does this.

> **Note:** Since Gemini returns a Pydantic model, you *could* use Pydantic methods to handle parsing.  
> We avoid that here to keep the workflow generally compatible across models.


## Erklärung zu Code-Block (Zelle 20)

**Was passiert hier?**
- - Ein JSON-String wird in ein Python-Objekt (meist dict/list) umgewandelt.
- - Anschließend wird das Objekt angezeigt/weiter genutzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Damit man Felder gezielt auslesen kann (z.B. `output['background']...`).
- - Ermöglicht einfache Weiterverarbeitung wie Auswertungen/Regeln/Alerts.

**Rolle im Gesamtprozess**
Auswertung: Antwort strukturieren, damit man damit arbeiten kann.

**Wichtig / typische Stolpersteine (optional)**
- Wenn der String kein valides JSON ist, gibt es einen Parsing-Fehler.


In [38]:
import re
import json 
def parse_json_in_output(output):
    """
    Extracts and converts JSON-like data from the given text output to a Python dictionary.
    
    Args:
        output (str): The text output containing the JSON data.
    
    Returns:
        dict: The parsed JSON data as a Python dictionary.
    """
    # Regex to extract JSON-like portion
    json_match = re.search(r"\{.*?\}", output, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        # Fix single quotes and ensure proper JSON formatting
        json_str = json_str.replace("'", '"')  # Replace single quotes with double quotes
        try:
            # Convert the fixed JSON string into a dictionary
            json_data = json.loads(json_str)  # JSON-String → Python-Objekt
            return json_data
        except json.JSONDecodeError:
            return "The extracted JSON is still not valid after formatting."
    else:
        return "No JSON data found in the given output."

## Erklärung zu Code-Block (Zelle 21)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [ ]:
#print(parse_json_in_output(response.text))


## Erklärung zu Code-Block (Zelle 22)

**Was passiert hier?**
- - Ein JSON-String wird in ein Python-Objekt (meist dict/list) umgewandelt.
- - Anschließend wird das Objekt angezeigt/weiter genutzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Damit man Felder gezielt auslesen kann (z.B. `output['background']...`).
- - Ermöglicht einfache Weiterverarbeitung wie Auswertungen/Regeln/Alerts.

**Rolle im Gesamtprozess**
Auswertung: Antwort strukturieren, damit man damit arbeiten kann.

**Wichtig / typische Stolpersteine (optional)**
- Wenn der String kein valides JSON ist, gibt es einen Parsing-Fehler.


In [39]:
json.loads(response.text)  # JSON-String → Python-Objekt


{'patientName': 'Claudie Fischer',
 'doctorName': 'Dr. Markus Müller',
 'medications': ['Ibuprofen', '400mg', '3x', 'nach dem Essen'],
 'dateOfBirth': '01.04.1978',
 'gender': 'f',
 'diagnosis': None,
 'signature': 'Reichmüller'}

Now let's do the same with GPT

## Erklärung zu Code-Block (Zelle 23)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [ ]:
im = "images/prescription1.jpg"

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",  # ausgewähltes Modell
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",  # Bild als Base64-Data-URL mitsenden
                    "image_url": {  # Bild als Base64-Data-URL mitsenden
                        "url": f"data:image/jpeg;base64,{encode_image(im)}",
                        #"detail": "low" -> je tiefer desto weniger tokens werden verwendet
                    }
                },
            ]}
    ],
    response_format={  # Ausgabeformat erzwingen (JSON)
                "type": "json_schema",   "json_schema": json_schema_prescription},
    temperature = 0  # Kreativität/Determinismus steuern
)

returnValue = completion.choices[0].message.content  # Antworttext aus Response extrahieren


## Erklärung zu Code-Block (Zelle 24)

**Was passiert hier?**
- - Der Code führt einen Zwischenschritt im Notebook aus (z.B. Aufruf, Parsing oder Anzeige).

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Unterstützt das iterative Experimentieren mit Vision-LLMs (Input → Output → prüfen).

**Rolle im Gesamtprozess**
Zwischenschritt im Experiment-Workflow.


In [41]:
returnValue

'{"doctor_name":"Dr. Markus Müller","patient_name":"Claudia Fischer","patient_dob":"1.4.1978","meds":[{"name":"Ibuprofen","dose":"400 mg","frequency":"3x","instructions":"nach dem Essen"}],"signature":true}'

Any difference wiht the output of Gemini vs your schema? 

No need for parsing now. We load the json in a python dict structure with json.loads

## Erklärung zu Code-Block (Zelle 25)

**Was passiert hier?**
- - Ein JSON-String wird in ein Python-Objekt (meist dict/list) umgewandelt.
- - Anschließend wird das Objekt angezeigt/weiter genutzt.

**Warum macht man das? (Data-Science/LLM-Kontext)**
- - Damit man Felder gezielt auslesen kann (z.B. `output['background']...`).
- - Ermöglicht einfache Weiterverarbeitung wie Auswertungen/Regeln/Alerts.

**Rolle im Gesamtprozess**
Auswertung: Antwort strukturieren, damit man damit arbeiten kann.

**Wichtig / typische Stolpersteine (optional)**
- Wenn der String kein valides JSON ist, gibt es einen Parsing-Fehler.


In [42]:
print(json.loads(returnValue))  # JSON-String → Python-Objekt


{'doctor_name': 'Dr. Markus Müller', 'patient_name': 'Claudia Fischer', 'patient_dob': '1.4.1978', 'meds': [{'name': 'Ibuprofen', 'dose': '400 mg', 'frequency': '3x', 'instructions': 'nach dem Essen'}], 'signature': True}


## Zusammenfassung für die Prüfung (max. 40 Zeilen)

Dieses Notebook zeigt die Grundidee von **Vision-Language Models (VLMs)**: Du kombinierst **Text (Prompt)** und **Bild** als Input, schickst beides an ein Modell (z.B. über OpenAI oder Gemini), bekommst ein **Response-Objekt** zurück und extrahierst daraus den eigentlichen Text/Output.

Der typische Ablauf ist: **Setup** (Imports, API-Key via `.env`, Client) → **Input vorbereiten** (Bildpfad laden, Bild in Base64 oder als Objekt übergeben) → **Model-Call** (`chat.completions.create` / `generate_content`) → **Output nutzen** (anzeigen, in JSON erzwingen, parsen, gezielt Felder auslesen).

Für Prüfungen wichtig: **Prompt ≠ Antwort**. Der Prompt enthält Ziel, Kontext und Formatvorgaben (z.B. „gib JSON zurück“). Unterschiedliche Prompts führen zu unterschiedlichen Antworten – deshalb testet man Prompts iterativ. Wenn im Call ein Chat-Verlauf (`messages`) verwendet wird, beeinflussen Rollen und Kontext (system/user/assistant) die Antwort.

Im Data-Science-Kontext ist die **Auswertung** zentral: Outputs sind nicht automatisch „wahr“. Du prüfst Plausibilität, Konsistenz und Schema (z.B. ob JSON wirklich valide ist). Für Weiterverarbeitung ist strukturierter Output (JSON) oft besser als Fließtext.

Typische Stolpersteine: fehlende API-Keys/ENV (Client kann nicht authentifizieren), falsche Modellnamen, **Response-Objekt vs. Text** (wo liegt der Inhalt?), sowie Parsing-Probleme bei JSON (`json.loads` schlägt fehl, wenn kein valides JSON zurückkommt). Auch Token-/Längenlimits können Antworten abschneiden.


## Mögliche Prüfungsfragen zu diesem Notebook (Einschätzung)

1. Erkläre den Zweck von `load_dotenv()` und warum API-Keys nicht direkt im Notebook stehen sollten.
2. Warum wird ein Bild für den API-Call oft als Base64-String kodiert? Was passiert dabei grob?
3. Erkläre den Aufbau von `messages`: Wie werden Text und Bild zusammen an das Modell übergeben?
4. Was ist der Unterschied zwischen einem „freien“ Bild-Describe-Call und einem Call mit `response_format={"type":"json_object"}`?
5. Warum kann `json.loads(...)` fehlschlagen, obwohl das Modell „JSON“ liefern sollte?
6. Wo findest du im Response-Objekt typischerweise den eigentlichen Antworttext?
7. Was bewirkt `temperature=0` in solchen Extraktionsaufgaben?
8. Nenne typische Risiken bei VLM-Ausgaben (z.B. Halluzinationen) und wie du sie im Workflow prüfst.
9. Was würde passieren, wenn der Bildpfad `img` falsch ist oder die Datei fehlt?
10. Wie würdest du Prompt A vs. Prompt B vergleichen, wenn beide ein JSON-Schema erfüllen sollen?
